<h2> RF </h2>
Trains a Random Forest model to classify stringency

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import mapclassify

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score, balanced_accuracy_score, classification_report

from itertools import product
import pickle
from pathlib import Path

from datetime import datetime, timedelta

from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import OneHotEncoder

import warnings

In [2]:
# Ignore a specific warning
warnings.filterwarnings('ignore', message='parsing')

In [3]:
# RF params

random_state = 0
min_samples_leaf = 15
max_depth = 50
n_classes = 3

# target y for prediction, e.g., low, medium or very high pollen
ytarget = 'stringency_type_int'
column = 'stringency'
area = 'whole'  # 'nl', 'all'
dic_labels = {0: 'low', 1: 'medium', 2: 'high'}
dic_labels_inv = {'low': 1, 'medium': 2, 'high': 3}

In [4]:
dfa = pd.read_csv('Training.csv', sep=';', parse_dates=['Date_reported'])
dfa

,Date_reported,Country,Cases per 100000,Deaths per 100000,ICU per 100000,stringency
0,2020-01-04,Austria,"7,524952338","0,235505215","2,411124818","81,48"
1,2020-02-04,Austria,"6,179208254","0,201861613","2,455982954","81,48"
2,2020-03-04,Austria,"6,156779186","0,123359874","2,747560839","81,48"
3,2020-04-04,Austria,"4,911965908","0,224290681","2,747560839","81,48"
4,2020-05-04,Austria,"4,620388023","0,201861613","2,736346305","81,48"
...,...,...,...,...,...,...
5638,2021-09-27,UK,"49,19051878","0,09391771","1,23881932","41,2"
5639,2021-09-28,UK,"55,88103757","0,059630292","1,231365534","41,2"
5640,2021-09-29,UK,"51,46094216","0,24895647","1,211985689","41,2"
5641,2021-09-30,UK,"52,26446035","0,223613596","1,214967203","41,2"


In [5]:
training_filename = 'Netherlands.csv'
df = pd.read_csv(training_filename, sep=';', parse_dates=['Date_reported'])
df.columns = 'Date_reported', 'Cases per 100000', 'Deaths per 100000', 'ICU per 100000', 'stringency'
df['Country'] = 'Netherlands'
df

,Date_reported,Cases per 100000,Deaths per 100000,ICU per 100000,stringency,Country
0,2020-02-27,0,0,"0,040137615","5,56",Netherlands
1,2020-02-28,"0,005733945",0,"0,04587156","5,56",Netherlands
2,2020-02-29,0,0,"0,051605505","5,56",Netherlands
3,2020-01-03,"0,005733945",0,"0,063073394","5,56",Netherlands
4,2020-02-03,"0,017201835",0,"0,05733945","5,56",Netherlands
...,...,...,...,...,...,...
578,2021-09-27,"9,174311927","0,005733945","0,940366972","41,67",Netherlands
579,2021-09-28,"7,964449541","0,02293578","0,883027523","41,67",Netherlands
580,2021-09-29,"9,850917431","0,04587156","0,819954128","41,67",Netherlands
581,2021-09-30,"10,04587156","0,03440367","0,802752294","41,67",Netherlands


In [6]:
def get_input_data(area):
    print('-------------' + area)
    training_filename = 'Training.csv'
    df = pd.read_csv(training_filename, sep=';', parse_dates=['Date_reported'])
    
    if (area == 'nl'):
        training_filename = 'Netherlands.csv'
        df = pd.read_csv(training_filename, sep=';', parse_dates=['Date_reported'])
        df.columns = 'Date_reported','Cases per 100000', 'Deaths per 100000', 'ICU per 100000', 'stringency'
        df['Country'] = 'Netherlands'
    # shape
    print(training_filename)
    
    # checkpoint null values
    print(df[column].isna().sum())
    
    # quick look at the data
    # df.tail()
    return df


In [7]:
def scale_df(df):
    
    # define the var to scale
    var_to_scale = ['Cases per 100000','Deaths per 100000','ICU per 100000']
    dtypes = {'Cases per 100000':float,'Deaths per 100000':float,'ICU per 100000':float, ytarget:int}
    # define a transformer
    minmax_transformer = Pipeline(steps=[('minmax', preprocessing.MinMaxScaler())])

    preprocessor = ColumnTransformer(
        remainder='passthrough', #passthough features not listed
        transformers=[('mm', minmax_transformer, var_to_scale)])
    Data = preprocessor.fit_transform(df)
    new_names = preprocessor.get_feature_names_out()
    original_names = [s.replace('mm__','') for s in new_names ]
    original_names = [s.replace('remainder__','') for s in original_names ]
    Xscaled = pd.DataFrame(data=Data, columns=original_names)
    Xscaled = Xscaled.astype({'Cases per 100000':float, 'Deaths per 100000': float, 'ICU per 100000': float,ytarget: int})
    return Xscaled

def preprocess_data(df):

    # change datatype from string to float
    df['stringency'] = df['stringency'].apply(lambda x: float(x.replace(',', '.')))
    df['Cases per 100000'] = df['Cases per 100000'].apply(lambda x: float(x.replace(',', '.')))

    df['Deaths per 100000'] = df['Deaths per 100000'].apply(lambda x: float(x.replace(',', '.')))

    df['ICU per 100000'] = df['ICU per 100000'].apply(lambda x: float(x.replace(',', '.')))
    df = df.astype({'Cases per 100000':float, 'Deaths per 100000': float, 'ICU per 100000': float,'stringency': float})

    # classify stringency - natural breaks
    np.random.seed(7)
    breaks = list(np.unique(np.insert(mapclassify.NaturalBreaks(df[column], k=n_classes).bins, 0, 0)))
    print(breaks)
    df[column + '_type_bin'] = pd.cut(df[column].copy(), bins=breaks, labels=False, include_lowest=True)
    df[column + '_type'] = df[column + '_type_bin'].map(dic_labels)
    df.drop(columns=[column + '_type_bin'], inplace=True)
    
    # inv map to
    df[ytarget] = df[column + '_type'].map(dic_labels_inv)
    
    # ensure it is an int
    df[ytarget] = df[ytarget].apply(lambda x: int(x))
    
    # print unique values
    print(df[ytarget].unique())

    # scale
    df.reset_index(drop=True, inplace=True)
    Xscaled = scale_df(df)
    
    return(Xscaled)

In [8]:
def get_train_test_all_nl(dfa, dfnl):
    
    # The al data will be used for training and the NL for testing
    df_train = dfa
    y_train = df_train[ytarget].to_numpy().ravel()
    df_test = dfnl
    y_test = df_test[ytarget].to_numpy().ravel()
    print('Number of records for training and test ', df_train.shape[0], df_test.shape[0])

    return df_train, y_train, df_test, y_test

In [9]:
def get_train_test(df):
    
    # print('dtypes in get_train_test', df.dtypes)
    ndays = (df.Date_reported.max() - df.Date_reported.min()).days
    ndays_tr = int(0.66*ndays)
    
    date_limit = df.Date_reported.min() + timedelta(days=ndays_tr)
    df_train = df.loc[df.Date_reported <= date_limit].copy(deep=True)
    y_train = df_train[ytarget].to_numpy().ravel()

    df_test = df.loc[df.Date_reported >  date_limit].copy(deep=True)
    y_test = df_test[ytarget].to_numpy().ravel()
    
    print('Number of records for training and test ', df_train.shape[0], df_test.shape[0])

    return df_train, y_train, df_test, y_test

In [10]:
def get_metrics(Y_true, Y_pred):

    balanced_OA = balanced_accuracy_score(Y_true, Y_pred, adjusted=True)
    # auc = roc_auc_score(Y_true, Y_pred, average= 'weighted')  - only one class is defined as ytrue (the value of the year)
    
    # true class will be only one, ypred might be different
    pred_class = np.unique(Y_pred)
    ytrue_class = np.unique(Y_true)
    all_class = list(set(np.concatenate([pred_class, ytrue_class])))
    
    creport = classification_report(Y_true, Y_pred, target_names = [x for x in dic_labels_inv if (dic_labels_inv[x] in all_class)])

    return balanced_OA, creport

In [11]:
def rf(X_train,  y_train, X_test, y_test):

    # Define the features for classification:

    exclude = ['Date_reported', 'Country', 'stringency', 'stringency_type', ytarget]
    variables = [x for x in X_train.columns if x not in exclude]
    # print(sorted(variables))

    # Prepare input data for RF - only get the selected feats
    X_train_feats = pd.DataFrame(data = X_train[variables])
    X_test_feats = pd.DataFrame(data = X_test[variables])

    # checkpoint
    print("X_train  ########## - y_train  ##########", X_train_feats.shape, y_train.shape)
    # print("X_train", X_train_feats.columns)
    print("X_test  ##########- y_test  ##########", X_test_feats.shape, y_test.shape)
    # print("X test", X_test_feats.columns)

    # define a random forest classifier, predict and get the metrics
    
    rf = RandomForestClassifier(n_estimators = 300, random_state = random_state,
                                max_depth = max_depth, min_samples_leaf = min_samples_leaf, n_jobs = -1)

    rf_classifier = rf.fit(X_train_feats, y_train)

    # get the feature importance
    importances = pd.Series(rf_classifier.feature_importances_, index=variables)
    
    # prediction
    yhat_tr = rf_classifier.predict(X_train_feats)
    yhat_ts = rf_classifier.predict(X_test_feats)

    # compute metrics over Tr and Ts

    OA_tr = accuracy_score(y_train, yhat_tr)
    OA_ts = accuracy_score(y_test, yhat_ts)
    balanced_OA, creport = get_metrics(y_test, yhat_ts)
    print ("RF metrics (validation) RF: OA: {:.2f}, Train_OA: {:.2f}".format(OA_ts, OA_tr))

    # create series
    yseries = pd.concat( [pd.Series(data=y_test, name ='ytrue'),pd.Series(data=yhat_ts, name = 'yhat')], axis=1)
    
    cols= X_test.columns
    result = pd.concat( [X_test[cols].reset_index(drop=True), yseries.reset_index(drop=True)], axis=1)  
    # print('result', result)
    # save prediction/model and importances to a csv file
    predict_filename = 'rf_class' + area + '.csv'
    model_name = 'rf_class_' +  area + '.pkl'
    result.to_csv(predict_filename, index=False)
    importances.to_csv('importances_' + area  + '.csv')

    # save the model
    with open(model_name, 'wb') as model_file:
        pickle.dump(rf_classifier, model_file)
    print('Done for ' + area)

    return OA_ts, OA_tr, creport

## Execute RF classifier

In [13]:
# defining data

results = []

# run classification with a 1/3 for testing
dfa = get_input_data('all')
dfta = preprocess_data(dfa)
dfnl = get_input_data('nl')
dftnl = preprocess_data(dfnl)

X_train,  y_train, X_test, y_test = get_train_test_all_nl(dfta, dftnl)

OA_ts, OA_tr, creport = rf(X_train,  y_train, X_test, y_test)
results.append({ 'OA_ts': OA_ts, 'OA_tr': OA_tr, 'creport': creport})

df_results = pd.DataFrame.from_dict(results)
metric_output_file = 'metrics_stringency_new.csv'
df_results.to_csv(metric_output_file, index=False)

print('.........done!')

-------------all
Training.csv
0
[0.0, 52.78, 70.37, 90.74]
[3 2 1]
-------------nl
Netherlands.csv
0
[0.0, 48.15, 67.59, 82.41]
[1 2 3]
Number of records for training and test  5643 583
X_train  ########## - y_train  ########## (5643, 3) (5643,)
X_test  ##########- y_test  ########## (583, 3) (583,)
RF metrics (validation) RF: OA: 0.57, Train_OA: 0.77
Done for whole
.........done!


In [14]:
print("done...")

done...
